In [2]:
source("utils/plot.R")

In [3]:
data <- read_tsv(snakemake@input$chrom, col_types=cols()) %>%
    mutate(middle = (start + end)/2) %>%
    select(!c(start, end)) %>%
    pivot_longer(cols=!c(middle, window_size), names_to="Map") %>%
    mutate(value=ifelse(value == 0, NA, value)) %>%
    filter(!(window_size==1e4 & !between(middle, snakemake@config$sweep_region_start, snakemake@config$sweep_region_end)))

In [6]:
data

In [12]:
sweep_loc = (snakemake@config$sweep_region_end + snakemake@config$sweep_region_start)/2

In [13]:
fig <- ggplot(data) +
    geom_vline(aes(xintercept=sweep_loc/1000), linetype='dashed', colour='darkgrey') +
    geom_line(aes(x=middle/1000, y=value, colour=Map)) +
    facet_wrap(vars(paste0(scales::comma(window_size/1000), 'kb windows')), nrow=2, scales='free') +
    turkana_colour +
    labs(x='Position (kb)', y='Rec. rate') +
    scale_x_continuous(labels=scales::comma) +
    scale_y_log10() +
    turkana_theme

turkana_save(snakemake@output[[1]], fig, asp=1.5, width=6)